In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor, kernels
from sklearn.gaussian_process.kernels import RBF
from sklearn.preprocessing import StandardScaler
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
ROOT = ".."
FILENAME = f"{ROOT}/data_calculated/02x_m_sin5w.csv"
RANDOM_STATE_TRAINTEST = 3 # 初期観測データ選択
RANDOM_STATE_TS = 2  # seed for TS
V = 0.3

In [ ]:
# データ取得
g_df = pd.read_csv(FILENAME)
DESCRIPTOR_NAMES = ['x1']
#DESCRIPTOR_NAMES = ['x1', 'x2']
#DESCRIPTOR_NAMES = ['x1','x2','x3' ]
#DESCRIPTOR_NAMES = ['x1','x2','x3','x4']
TARGET_NAME = "y"

In [ ]:
g_df.plot(x="x1",y="y")

In [ ]:
g_Xraw = g_df[DESCRIPTOR_NAMES].values
g_y = g_df[TARGET_NAME].values
g_y = g_y - np.mean(g_y)
# データ加工
g_scaler = StandardScaler()
g_X = g_scaler.fit_transform(g_Xraw)

In [ ]:
from BO_misc import plot_GPR
def search_candidate_UCB(it, train, X, y, reg, v, filename=None):
    """search next action in the UCB method

    Args:
        it (int): the number of iteration
        train (np.array): the index of training data
        X (np.array): descriptor
        y (np.array): target values
        reg (regressor): regressor
        v (float): a factor of sqrt(v*it)*stddev
        filename (str, optional): filename. Defaults to None.

    Returns:
        int: next action
    """
    # GPR training data setの作成
    Xtrain = X[train]
    ytrain = y[train]
    reg.fit(Xtrain, ytrain)
    print("kernel=", reg.kernel_)
    yp_mean, yp_std = reg.predict(X, return_std=True)
    acq = yp_mean + yp_std*np.sqrt(v*it)
    ia = np.argmax(acq)
    plot_GPR(X[:,0], y, Xtrain, ytrain, yp_mean, yp_std, acq, 
             it, ia=ia)
    return ia

In [ ]:
def make_model(optimize=False):
    if optimize:
        kernel = RBF(length_scale=1)
        reg = GaussianProcessRegressor(kernel=kernel)
    else:
        kernel = RBF(length_scale=1)
        reg = GaussianProcessRegressor(kernel=kernel, optimizer=None)
    return reg

In [ ]:
def get_train(nall, nchoice=2, seed_initial_selection=0):
    # データ解析 (simulationを行う．)
    random.seed(seed_initial_selection)
    idx = range(nall)
    train = random.sample(idx, nchoice)
    return train


g_reg = make_model()

g_train = get_train(g_X.shape[0], 3, RANDOM_STATE_TRAINTEST)

print("train", g_train)
for _it in range(8):
    print("\niteration=", _it+1)

    g_ia = search_candidate_UCB(
        _it, g_train, g_X, g_y, g_reg, V)
    print("next action=", g_ia, "x=", g_X[g_ia, 0])
    g_train = np.hstack([g_train, g_ia])
    print("action=", g_train)

In [ ]:
g_train = get_train(g_X.shape[0], 3, RANDOM_STATE_TRAINTEST) #　訓練データを得る

USE_SAMPLE_Y = True # ガウス過程.sample_y()を使うか、multivariate_normal.rvs()を使うか

g_Xtrain = g_X[g_train] # 訓練データX
g_ytrain = g_y[g_train] # 訓練データy
g_reg.fit(g_Xtrain, g_ytrain) # 学習

if USE_SAMPLE_Y:
    g_yp_mean = g_reg.predict(g_X) # 平均値のみ
else:
    print("calculate covariance matrix")
    g_yp_mean, g_yp_covarencematrix = g_reg.predict(g_X, return_cov=True) # 平均値と共分散行列

In [ ]:
from scipy.stats import multivariate_normal
g_fig, g_ax = plt.subplots()
# 50\vec{Y}点選択
for _i in range(50): 
    if USE_SAMPLE_Y:
        # random_stateの変数を変えないと、全て同じg_acqが得られる。
        g_acq = g_reg.sample_y(g_X, random_state=RANDOM_STATE_TS+_i)
    else:
        # 平均値と共分散行列を顕に用いる。
        g_acq = multivariate_normal.rvs(g_yp_mean, g_yp_covarencematrix)
    g_ax.plot(g_X[:, 0], g_acq, color="red", alpha=0.1)
g_ax.plot(g_X[:, 0], g_y, "--", color="blue")  # ,label="expriment")
g_ax.plot(g_Xtrain[:, 0], g_ytrain, "o", color="blue", label="expriment")
# 赤線が50 lines描かれる。

In [ ]:
g_iopt = np.argmax(g_y) # 最適値のindex
print("iopt", g_iopt)

In [ ]:
def search_candidate_TS(it, train, X, y, reg, random_state=0):
    """search next action in the TS method
    
    acq = reg.sample_y(X, random_state=it+random_state) is used.

    Args:
        it (int): the number of iteration. This is a dummy.
        train (np.array): the index of training data
        X (np.array): descriptor
        y (np.array): target values
        reg (GaussianProcessRegressor): regressor
        random_state (int): random state for reg.sample_y(). Defaults to 0

    Returns:
        int: next action
    """
    # GPR training data setの作成
    Xtrain = X[train]
    ytrain = y[train]
    reg.fit(Xtrain, ytrain)
    print("kernel=", reg.kernel_)
    yp_mean, yp_std = reg.predict(X, return_std=True)
    acq = reg.sample_y(X, random_state=it+random_state)
    ia = np.argmax(acq)
    plot_GPR(X[:,0], y, Xtrain, ytrain, yp_mean, yp_std, acq, 
             it, ia=ia)
    return ia

In [ ]:
g_train = get_train(g_X.shape[0], 3, RANDOM_STATE_TRAINTEST)
g_reg = GaussianProcessRegressor()
for it in range(10):
    print("iteration=", it+1)
    print("train=", g_train)
    g_ia = search_candidate_TS(it, g_train, g_X, g_y, g_reg, 
                             random_state=RANDOM_STATE_TS+it)
    print("next action=", g_ia, "x=", g_X[g_ia, 0])
    g_train = np.hstack([g_train, g_ia])
    if g_iopt == g_ia:
        break